# Import from kaggle (Kagglehub provided code)

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/skin-cancer-mnist-ham10000


# Read and build model

In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
from pathlib import Path

# Config
# Same as S-Net paper
IMG_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 80
DATASET_ROOT = Path(path)
# 2 folders from dataset
IMAGE_PART_1 = DATASET_ROOT / "HAM10000_images_part_1"
IMAGE_PART_2 = DATASET_ROOT / "HAM10000_images_part_2"

# Load data
# Full name for each label
DX_LABEL_MAP = {
    'nv': "Melanocytic Nevus",
    'mel': "Melanoma",
    'bkl': "Benign Keratosis",
    'bcc': "Basal Cell Carcinoma",
    'akiec': "Actinic Keratoses",
    'vasc': "Vascular Lesion",
    'df': "Dermatofibroma"
}

# Load CSV
df = pd.read_csv(DATASET_ROOT / "HAM10000_metadata.csv")
df['diagnosis_full'] = df['dx'].map(DX_LABEL_MAP)

# Map label names to integers
class_names = sorted(df['diagnosis_full'].unique())
# for name in class names
label_to_idx = {name: i for i, name in enumerate(class_names)}
df['label_idx'] = df['diagnosis_full'].map(label_to_idx)

# Find image path from both folders
def find_image_path(image_id):
    for folder in [IMAGE_PART_1, IMAGE_PART_2]:
        path = folder / f"{image_id}.jpg"
        if path.exists():
            return path
    return None

# apply image map
df["image_path"] = df["image_id"].apply(find_image_path)
df = df[df["image_path"].notnull()]  # filter missing

# Load image and Label

images, labels = [], []
print("Loading images...")
# for col, row in image pixel
for _, row in tqdm(df.iterrows(), total=len(df)):
    img = Image.open(row["image_path"]).convert("RGB").resize((IMG_SIZE, IMG_SIZE))
    images.append(np.array(img) / 255.0)
    labels.append(row["label_idx"])

images = np.array(images, dtype=np.float32)
labels = to_categorical(labels, num_classes = len(class_names))

# Split and convert to tf.data

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size = 0.2, stratify=np.argmax(labels, axis = 1), random_state = 42)

# tensor slice data
train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

NUM_CLASSES = len(class_names)
print(f"Detected classes: {class_names}")

# Build the shallow model (S-Net)

# Same as the paper
def shallow_model(input_shape = (IMG_SIZE, IMG_SIZE, 3), num_classes = NUM_CLASSES):
    inputs = layers.Input(shape=input_shape)

    # Block 1 module
    x = layers.Conv2D(64, 5, padding = 'same', activation = 'relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, 3, padding = 'same', activation = 'relu')(x)
    x = layers.BatchNormalization()(x)
    feat1 = x

    x = layers.MaxPooling2D(pool_size=(2, 2))(feat1)

    # Block 2 module
    x = layers.Conv2D(128, 5, padding = 'same', activation = 'relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(128, 3, padding = 'same', activation = 'relu')(x)
    x = layers.BatchNormalization()(x)
    feat2 = x

    gap1 = layers.GlobalAveragePooling2D()(feat1)
    gap2 = layers.GlobalAveragePooling2D()(feat2)
    merged = layers.concatenate([gap1, gap2])
    merged = layers.Dropout(0.5)(merged)

    outputs = layers.Dense(num_classes, activation = 'softmax')(merged)
    return models.Model(inputs, outputs)

model = shallow_model()
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.summary()

# Train

history = model.fit(train, validation_data = val, epochs = EPOCHS)

# Show training

plt.plot(history.history['accuracy'], label = 'Train')
plt.plot(history.history['val_accuracy'], label = 'Val')
plt.title("Training Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# save model
model.save("shallow_skin_model.h5")

# Predict function will be used for single image when connect to interface

def predict_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size = (IMG_SIZE, IMG_SIZE))
    img = tf.keras.utils.img_to_array(img) / 255.0
    img = tf.expand_dims(img, axis = 0)
    preds = model.predict(img)[0]
    pred_idx = tf.argmax(preds).numpy()
    pred_class = class_names[pred_idx]
    confidence = tf.reduce_max(preds).numpy()
    print(f"Prediction: {pred_class} ({confidence:.2%})")



Loading images...


100%|██████████| 10015/10015 [02:35<00:00, 64.57it/s]


Detected classes: ['Actinic Keratoses', 'Basal Cell Carcinoma', 'Benign Keratosis', 'Dermatofibroma', 'Melanocytic Nevus', 'Melanoma', 'Vascular Lesion']


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 224, 224,  │      4,864 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 224, 224,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 224, 224,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 112, 112,  │    204,928 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 112, 112,  │    147,584 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 128)       │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 192)       │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 192)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 7)         │      1,351 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 397,191 (1.52 MB)

 Trainable params: 396,423 (1.51 MB)

 Non-trainable params: 768 (3.00 KB)

Epoch 1/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 35s 40ms/step - accuracy: 0.5037 - loss: 1.4581 - val_accuracy: 0.6690 - val_loss: 0.9740
Epoch 2/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6750 - loss: 0.9233 - val_accuracy: 0.3525 - val_loss: 1.7217
Epoch 3/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6819 - loss: 0.8957 - val_accuracy: 0.1543 - val_loss: 2.6932
Epoch 4/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6782 - loss: 0.8951 - val_accuracy: 0.1118 - val_loss: 11.1204
Epoch 5/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6894 - loss: 0.8328 - val_accuracy: 0.1478 - val_loss: 5.6512
Epoch 6/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6965 - loss: 0.8402 - val_accuracy: 0.4618 - val_loss: 1.7781
Epoch 7/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.7066 - loss: 0.8112 - val_accuracy: 0.6780 - val_loss: 1.0054
Epoch 8/80
501/501 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.6999 - loss: 0.8036 -

# Check Validation Accuracy

In [ ]:
# Get predictions on the validation set
y_pred_probs = model.predict(X_val, batch_size=BATCH_SIZE)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = np.argmax(y_val, axis=1)

# Save as val when changing the modle above
val_images = X_val

# Map class index to label name
idx_to_label = {i: name for i, name in enumerate(class_names)}

# Sample 25 indices randomly
sample_indices = random.sample(range(len(val_images)), 25)

# Plot predictions
plt.figure(figsize=(15, 15))
for i, idx in enumerate(sample_indices):
    image = val_images[idx]
    true_label = idx_to_label[y_true[idx]]
    pred_label = idx_to_label[y_pred[idx]]
    confidence = y_pred_probs[idx][y_pred[idx]]
    correct = (y_true[idx] == y_pred[idx])

    plt.subplot(5, 5, i + 1)
    plt.imshow(image)
    plt.axis('off')
    title_color = 'green' if correct else 'red'
    plt.title(
        f"Pred: {pred_label} ({confidence:.1%})\nTrue: {true_label}",
        color=title_color,
        fontsize=9
    )

plt.suptitle("Validation Images with Predicted vs Ground Truth Labels", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.92)
plt.show()


In [ ]:
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_pre
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_pre
import time

# Apply model-specific preprocessing
X_val_mobilenet = mobilenet_pre(X_val.copy() * 255.0)
X_val_resnet = resnet_pre(X_val.copy() * 255.0)

# Helper function to build benchmarking models
def build_backbone_model(backbone_fn, preprocess_version = 'default'):
    base_model = backbone_fn(
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        weights = 'imagenet'
    )
    # No compare yet, just train
    base_model.trainable = False
    # Force each layer to be similar to my model
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

    # Construct model with similar input
    if preprocess_version == 'mobilenet':
        x = mobilenet_pre(inputs * 255.0)
    elif preprocess_version == 'resnet':
        x = resnet_pre(inputs * 255.0)
    else:
        x = inputs

    x = base_model(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(NUM_CLASSES, activation = 'softmax')(x)
    model = models.Model(inputs, outputs)
    return model

# Build models to compare with
mobilenet_model = build_backbone_model(MobileNetV2, preprocess_version = 'mobilenet')
resnet_model = build_backbone_model(ResNet50, preprocess_version = 'resnet')

# Run same epoch and optmizer
for model in [mobilenet_model, resnet_model]:
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy'])

# Train models
mobilenet_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)
resnet_model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Benchmark accuracy and throughput
def benchmark(model, X_val_input, y_val):
    # Accuracy
    loss, acc = model.evaluate(X_val_input, y_val, verbose = 0)

    # Throughput (inference on a 64-image batch)
    batch = X_val_input[:64]
    start = time.time()
    _ = model.predict(batch)
    # Check duration in terms of time stamp
    duration = time.time() - start
    # images per second
    throughput = 64 / duration

    return acc, throughput

# Benchmark each model
acc_snet, throughput_snet = benchmark(model, X_val, y_val)
acc_mobile, throughput_mobile = benchmark(mobilenet_model, X_val_mobilenet, y_val)
acc_resnet, throughput_resnet = benchmark(resnet_model, X_val_resnet, y_val)

# Print results
print("Benchmark Results (64-image batch):")
print(f"S-Net:        Accuracy = {acc_snet:.2%}, Throughput = {throughput_snet:.2f} img/s")
print(f"MobileNetV2:  Accuracy = {acc_mobile:.2%}, Throughput = {throughput_mobile:.2f} img/s")
print(f"ResNet50:     Accuracy = {acc_resnet:.2%}, Throughput = {throughput_resnet:.2f} img/s")


In [ ]:
# Same thing as above
# Random images
sample_indices = random.sample(range(len(X_val)), 25)

# For each model
y_pred_snet = np.argmax(model.predict(X_val, batch_size=BATCH_SIZE), axis=1)
y_pred_mobile = np.argmax(mobilenet_model.predict(X_val_mobilenet, batch_size=BATCH_SIZE), axis=1)
y_pred_resnet = np.argmax(resnet_model.predict(X_val_resnet, batch_size=BATCH_SIZE), axis=1)
# Ground truth
y_true = np.argmax(y_val, axis=1)

# Plot
fig, axes = plt.subplots(3, 25, figsize=(25, 6))

# Ensure label map
idx_to_label = {i: name for i, name in enumerate(class_names)}

# For each model
for col, idx in enumerate(sample_indices):
    image = X_val[idx]
    true_label = idx_to_label[y_true[idx]]

    # Snet
    pred_label_snet = idx_to_label[y_pred_snet[idx]]
    correct_snet = (y_true[idx] == y_pred_snet[idx])
    conf_snet = np.max(model.predict(X_val[idx:idx+1]))
    axes[0, col].imshow(image)
    axes[0, col].axis('off')
    axes[0, col].set_title(f"{pred_label_snet}\n({conf_snet:.0%})", color  = 'green' if correct_snet else 'red', fontsize=8)

    # MobileNetV2
    pred_label_mobile = idx_to_label[y_pred_mobile[idx]]
    correct_mobile = (y_true[idx] == y_pred_mobile[idx])
    conf_mobile = np.max(mobilenet_model.predict(X_val_mobilenet[idx:idx+1]))
    axes[1, col].imshow(image)
    axes[1, col].axis('off')
    axes[1, col].set_title(f"{pred_label_mobile}\n({conf_mobile:.0%})", color  = 'green' if correct_mobile else 'red', fontsize=8)

    # ResNet50
    pred_label_resnet = idx_to_label[y_pred_resnet[idx]]
    correct_resnet = (y_true[idx] == y_pred_resnet[idx])
    conf_resnet = np.max(resnet_model.predict(X_val_resnet[idx:idx+1]))
    axes[2, col].imshow(image)
    axes[2, col].axis('off')
    axes[2, col].set_title(f"{pred_label_resnet}\n({conf_resnet:.0%})", color  = 'green' if correct_resnet else 'red', fontsize=8)

# Set each model into each axis
axes[0, 0].set_ylabel("S-Net", fontsize=12)
axes[1, 0].set_ylabel("MobileNetV2", fontsize=12)
axes[2, 0].set_ylabel("ResNet50", fontsize=12)

# Plot model
plt.suptitle("Model Predictions on 25 Sample Validation Images\n(Green: Correct, Red: Incorrect)", fontsize=16)
plt.tight_layout()
plt.subplots_adjust(top=0.82)
plt.show()

# Ran the same thing again in the run, deleted the duplicated half for clarity.